1) 지역 특징 추출
    - sift 검출기 사용하여 각각의 영상에서 지역 특징점 검출
2) 대응점 매칭
    - 1에서 찾은 지역 특징점 간 매칭 수행
    - 매칭 전략
        - 고정 임계값 사용
        - 최근접 특징 벡터 검색 
        - 최근접 거리 비융(*실습에서 사용)
3) 변환 행렬(H) 추정
    - 영상 1과 영상 2 사이의 다수의 대응점 쌍으로부터 RANSAC을 이용하여 변환 행렬 추정
    - RANSAC은 이상점이 포함된 데이터셋에서 어떠한 모델을 예측할 때 효과적인 방법
4) 원근 변환
    - 추정된 변환 행렬 H를 이용하여 영상2에 원근 변환 적용
5) 두 영상 이어붙이기
    - 파노라마 결과 영상 = 영상1 + 변환된 영상2

In [2]:
import cv2
import numpy as np
## 테스트할 이미지
img_1 = cv2.imread('left.jpg')
img_1 = cv2.resize(img_1, None, fx=0.5, fy=0.5)
gray_1 = cv2.cvtColor(img_1,cv2.COLOR_BGR2GRAY)
img_2 = cv2.imread('right.jpg')
img_2 = cv2.resize(img_2, None, fx=0.5, fy=0.5)
gray_2 = cv2.cvtColor(img_2,cv2.COLOR_BGR2GRAY)

In [9]:
# 1. sift 이용해서 지역 특징 추출
sift = cv2.xfeatures2d.SIFT_create()
kp_1, des_1 = sift.detectAndCompute(gray_1,None) #KeyPoint(kp), 특징기술자(des) 뽑아줌 
kp_2, des_2 = sift.detectAndCompute(gray_2,None)

# 2.1 두 영상의 지역 특징 간 거리 계산
bf = cv2.BFMatcher()
matches =bf.knnMatch(queryDescriptors = des_1, trainDescriptors = des_2,k=2) # 특징점 하나당 거리가 가까운 상위 K개의 대응점 찾음

# 2.2 '최근접 거리 비융' 매칭 전략(ratio testing) 을 사용하여 대응점 쌍 생성
ratio = 0.7
good = []
for m,n in matches:
    if m.distance < n.distance * ratio:
        good.append(m)

In [10]:
## optional) SIFT 특징점 매칭 결과 확인하기
print(len(good))
matched = cv2.drawMatches(img1 = img_1, 
                          keypoints1 = kp_1, 
                          img2 = img_2, 
                          keypoints2 = kp_2, 
                          matches1to2 = good[:20],   # 첫 20개 매칭쌍만 시각화
                          outImg = None, 
                          flags = 2)

#cv2.imshow('matching result', matched)
#cv2.waitKey(0)

#cv2.destroyAllWindows()

243


In [11]:
# RANSAC을 이용한 변환 행렬 H 추정

## 참고. cv2.getPerspectiveTransform()와 cv2.findHomography()의 차이
##   cv2.getPerspectiveTransform() - 4개의 대응점 쌍을 입력하면 변환 행렬을 반환해줌
##   cv2.findHomography() - 4개 이상의 대응점 쌍을 입력하면 가장 대응점들을 가장 잘 만족하는 변환 행렬을 반환해줌

if len(good) >= 4:   # 4개 이상의 대응점이 존재해야 원근 변환 행렬을 추정 가능
    src_pts = np.float32([ kp_2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp_1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    H, _ = cv2.findHomography(srcPoints = src_pts, 
                              dstPoints = dst_pts, 
                              method = cv2.RANSAC, 
                              ransacReprojThreshold = 5) 
    
else:
    raise AssertionError("Can't find enough keypoints.")

In [ ]:
## 4) 원근 변환 적용
## 변환 행렬 H를 사용하여 변환 수행
## https://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html#warpperspective
## cv2.warpPerspective(src, M, dsize[, dst[, flags[, borderMode[, borderValue]]]]) → dst
## M -> H
## 오른쪽 영상 변환해서 왼쪽 사진에 이어 붙이기
#height1, width1 = img_1.shape[:2]
#height2, width2 = img_2.shape[:2]
#transformed = cv2.warpPerspective(img_2,H,(width1+width2,height1))

res = cv2.warpPerspective(src = img_2,
                          M = H,
                          dsize = (img_1.shape[1]+img_2.shape[1], img_1.shape[0]))

# cv2.imshow('right image', img_r)
# cv2.imshow('transformed right image', res)
# cv2.imshow('left image', img_l)

## 5) 두 영상 이어붙이기
#res = np.hstack((img_1,transformed))
res[0:img_1.shape[0], 0:img_1.shape[1]] = img_1 # 왼쪽 영상을 변환된 오른쪽 영상에 오버랩함
cv2.imshow('panorama', res)

cv2.waitKey(0)

cv2.destroyAllWindows()